# Thermodynamic analysis of strategy enabling anaerobic production of PHB

In this script, we have performed the MDF analyses for the thermodynamic investigation of the NOG+EMP pathway in comparison to other strategies enabling PHB production, also demonstrating how the analysis of other pathways and metabolic engineering strategies could be analyzed by other researchers with the custom-made functions we have developed in this study.

In [1]:
from equilibrator_api import Q_ #for creating quantity variables
import equilibrator_custom_functions as eq #custom functions created during analysis
import numpy as np #numerical package
import math
import matplotlib.pyplot as plt #plotting package

Fragments already downloaded
Fragments already downloaded


# Input Files and their formats

The script's input is based on the two custom __input csv files__, which are:

   __The compound file__, with metabolite names, IDs and bounds. It is currently name "metabolite_reference_table"
   
   This file uses a custom csv file format, which contains information of multiple metabolites, to create a dictionary which will be used for the eQuilibrator API for high throughput analysis. This dictionary allows for writing reactions in a simpler way (with acronyms), and customizing the ranges of metabolites according to data. The columns contain the following elements:

| Metabolite Acronym        | keggID                          | Lower Bound (M)             | Upper Bound (M)               |
|------------------|----------------------------------|----------------------|------------------------|
|Custom-defined name for metabolite| Kegg ID to identify metabolite (necessary to use specific kegg ID) | Custom-defined minimal concentration (in M)| Custom-defined maximal concentration (in M) |

Notice that the column names do not need to be mantained in order to be recognized in the script. More usefull information can be added to the other columns, as the script will ignore the information stored in these.

   __The pathway file__, with reaction names, formulae, fluxes, and, in the case of ECM, enzyme parameters (which will be read by other functions below). Being pathway specific, there are many files referenced in this script. The columns contain the follow basic setup:
   
| Reaction Formula        | Relative Flux                          | Reaction Name  |     Standard dG (kJ/mol)        | kcrf(1/s) | MWe(Da)        | kM(mM) |
|------------------|----------------------------------|----------------------|------------------------|-----------------|-------|-----|
|Using the acronyms defined in the compound file: S1 + S2 <=> P1 + P2| The relative stoichiometric coefficient of the reaction in the pathway | Customizable name to identify the reaction in graphs, for example | __(Optional)__ Standard Gibs Free Energy of Reaction in kJ/mol. These are custom values that can replace the values obtained from the eQuilibrator packages|__(Only needed for ECM)__ Forward catalytic rate of reaction's enzyme in 1/s (mind the direction of the reaction) |__(Only needed for ECM)__ Enzyme molecular weight, equivalent to Daltons |__(Only needed for ECM)__ Michaelis costants of each substrate and product (except water) in milimolar, with format example 'S1: 0.1 S2:0.02 P1:3 P2:0.3' | 

# Setting Intracellular Conditions

Below we set the intracellular conditions in *E. coli*, i.e the pH, ionic strengh, temperature and pMg (which have an effect on the thermodynamics of reactions). Note that metabolite concentration bounds are also organism specific, but are defined in the compound file.

In the cell bellow the name of the compound csv file is defined __("metabolite_reference_table")__. With the function __eq.obtain_compound_settings( string: compound_file_name )__, we construct a __CompoundSettings__ object containing data structures for idetifying compounds and setting their maximal and minimal concentrations for the model)

With __eq.change_bounds__ , one can create a new CompoundSettings object with specific changes in the metabolite concentration bounds.

The __eq.fix_concentrations__ function can also be used when the concentration needs to fixed at a specific concentration

In [2]:
#E. coli physiological conditions as a dictionary
E_coli_con = {
            "p_h": str(Q_("7.5")),
            "p_mg": str(Q_(3)),
            "ionic_strength": str(Q_('200 mM')),
            "temperature": str(Q_(37+273.15,"K")),
            "kcat_source": "fwd",
}

#Generating Compound Settings with the default eQuilibrator bounds
cs_default_bounds = eq.obtain_compound_settings("metabolite_reference_table_custom", custom_bounds = True)

# Initial Max-min Driving Force (MDF) analysis

The MDF analysis can be quickly performed with the __eq.MDF_analysis__ function, receiving a full report of the results:

In [3]:
#mdf_result = eq.MDF_analysis("EMP",cs_default_bounds,E_coli_con)
#import os  
#os.rename('MDF.tiff','MDFEMP_custom.tiff')

In [4]:
#mdf_result = eq.MDF_analysis("ED",cs_default_bounds,E_coli_con)
#os.rename('MDF.tiff','MDFED_custom.tiff')

In [ ]:
mdf_result = eq.MDF_analysis("NOGEMP",cs_default_bounds,E_coli_con)
#os.rename('MDF.tiff','MDFNOGEMP_custom.tiff')

In [ ]:
#mdf_result = eq.MDF_analysis("EMPsc",cs_default_bounds,E_coli_con)

In [ ]:
#mdf_result = eq.MDF_analysis("EDsc",cs_default_bounds,E_coli_con)

In [ ]:
#mdf_result = eq.MDF_analysis("NOGEMPsc",cs_default_bounds,E_coli_con)

In [ ]:
mdf_result = eq.MDF_analysis("NOGEMPsuc",cs_default_bounds,E_coli_con)

# Concentration Variability Analysis (CVA)

A function for easy execution of CVA has also been created, __eq.MDF_CVA__. The results will automatically be printed unless  __print_results = False__ is used. The optional argument __compounds__ can also be used to specify what metabolites will be analyzed, but by default CVA will be performed for all metabolites except water. The function will highlight which linear problems are unbounded (in which concentration could vary indefinitely) or unfeasible. CVA results are given in the form of a list of tuples with metabolite name, minimum and maximum concentrations and can be later plotted with __eq.CVA_plot__, for when the user would like to customize the plot layout:

In [ ]:
cva = eq.MDF_CVA(1,'EMP',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EMP_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(3,'EMP',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EMP_3kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(5,'EMP',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EMP_5kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_EMP_5kJ.csv')

In [ ]:
cva = eq.MDF_CVA(1,'ED',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_ED_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(5,'ED',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_ED_5kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_ED_5kJ.csv')

In [ ]:
cva = eq.MDF_CVA(1,'NOGEMP',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)

fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_NOGEMP_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(3,'NOGEMP',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_NOGEMP_3kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_NOGEMP_3kJ.csv')

In [ ]:
cva = eq.MDF_CVA(1,'EMPsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EMPsc_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(5,'EMPsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EMPsc_5kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_EMPsc_5kJ.csv')

In [ ]:
cva = eq.MDF_CVA(1,'EDsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EDsc_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(5,'EDsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_EDsc_5kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_EDsc_5kJ.csv')

In [ ]:
cva = eq.MDF_CVA(1,'NOGEMPsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)

fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_NOGEMPsc_1kJ.tiff', dpi = 150)

In [ ]:
cva = eq.MDF_CVA(3,'NOGEMPsc',cs_default_bounds,E_coli_con, custom_dGs = False, print_results = False)
fig = plt.figure(figsize=(12, 7.5))
ax = plt.subplot()
eq.CVA_plot(cva, ax=ax)

plt.rcParams['font.size'] = '16'

# Set tick font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)

plt.ylabel('Metabolite concentration (M)',fontsize=16)
#plt.xlabel('Metabolites',fontsize=16)
#ax.set_title('Concentration Variability Analysis for an MDF = 2.8 kJ/mol', fontsize=16)
plt.tight_layout()
print(cva)
fig.savefig('CVA_NOGEMPsc_3kJ.tiff', dpi = 150)

In [ ]:
import pandas as pd 
df = pd.DataFrame(cva)
df.to_csv('CVA_NOGEMPsc_3kJ.csv')